In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
glove_file = r"C:\Users\ajayp\OneDrive\Desktop\Deep Learning\DL Projects\3.Spam filter for Quora questions\P3 Quora Text Classification with embeddings\glove.6B.100d.txt"
embedding_dim = 100
validation_split = 0.2

In [3]:
dataset = r"C:\Users\ajayp\OneDrive\Desktop\Deep Learning\DL Projects\3.Spam filter for Quora questions\P3 Quora Text Classification with embeddings\train.csv"

In [4]:
data = pd.read_csv(dataset)


In [5]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [6]:
#dropping qid column
data.drop(['qid'],axis =1,inplace = True)

In [7]:
data 

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [8]:
#format text samples and labels into tensors that can be fed into neural network 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence


C:\Users\ajayp\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
tok = Tokenizer()
tok.fit_on_texts(data['question_text'])
sequences = tok.texts_to_sequences(data['question_text'])

In [10]:
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [11]:
#finding the max length of the sequence
max_len = 0
i = 0
for i in range (len(sequences)):
    if len (sequences[i]) > max_len:
        max_len = len(sequences[i])
print(max_len)

190


In [12]:
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [13]:
from keras.preprocessing.sequence import pad_sequences

In [14]:
X_data = pad_sequences(sequences, maxlen = max_len)

In [15]:
from keras.utils import to_categorical

In [16]:
labels = to_categorical(np.asarray(data['target']))
print(labels.shape)
print(X_data.shape)

(1306122, 2)
(1306122, 190)


In [17]:
#split the data into a training set and a validation set
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
labels = labels[indices]
nb_validation_samples = int(validation_split * X_data.shape[0])
print(nb_validation_samples)

261224


In [18]:
x_train = X_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [19]:
x_train

array([[     0,      0,      0, ...,     45,     55,    110],
       [     0,      0,      0, ...,  69622,  10329, 149915],
       [     0,      0,      0, ...,      7,  46688,  16637],
       ...,
       [     0,      0,      0, ...,    549,      6,     57],
       [     0,      0,      0, ...,    486,   6623,    228],
       [     0,      0,      0, ...,    303,   2635,    737]])

In [20]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [21]:
#preparing the embedding layer
embeddings_index = {}
f = glove_file
f = open(f,encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [22]:
#At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [23]:
embedding_matrix.shape

(222162, 100)

In [24]:
#We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [25]:
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

In [28]:
#Finally we can then build a small 1D convnet to solve our classification problem:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(2)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [29]:
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=5, batch_size=128)

Epoch 1/5


8164/8164 [==============================] - 694s 85ms/step - loss: 0.2025 - acc: 0.9425 - val_loss: 0.2002 - val_acc: 0.9438
Epoch 2/5
8164/8164 [==============================] - 680s 83ms/step - loss: 0.1978 - acc: 0.9437 - val_loss: 0.1968 - val_acc: 0.9443
Epoch 3/5
8164/8164 [==============================] - 689s 84ms/step - loss: 0.1957 - acc: 0.9444 - val_loss: 0.1974 - val_acc: 0.9441
Epoch 4/5
8164/8164 [==============================] - 727s 89ms/step - loss: 0.1936 - acc: 0.9452 - val_loss: 0.1993 - val_acc: 0.9444
Epoch 5/5
8164/8164 [==============================] - 713s 87ms/step - loss: 0.1918 - acc: 0.9459 - val_loss: 0.2029 - val_acc: 0.9439


In [30]:
#prediction on x_val
y_pred = model.predict(x_val)

8164/8164 [==============================] - 78s 9ms/step


In [31]:
y_pred

array([[0.95446855, 0.04546388],
       [0.95446855, 0.04546388],
       [0.95446855, 0.04546388],
       ...,
       [0.95446855, 0.04546388],
       [0.95446855, 0.04546389],
       [0.9663506 , 0.03420138]], dtype=float32)

In [32]:
from  sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val,y_pred)

In [33]:
y_val

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [34]:
auc

0.6708504466633048

In [35]:
y_pred [y_pred > 0.5] = 1.0
y_pred[y_pred <= 0.5] = 0.0

In [36]:
pred = pd.DataFrame(y_pred)

In [37]:
pred

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
261219,1.0,0.0
261220,1.0,0.0
261221,1.0,0.0
261222,1.0,0.0


In [38]:
pred.to_csv('predctions2.csv',index=False)